# Installation

Agent Framework has been installed in the previous step. If it has not been installed, please run the following command in the terminal to install it.

```
# Install core framework
pip install agent-framework
```

# Setup

Install the Microsoft Agent Framework packages. The framework is modular - you can install just what you need.

Import required libraries for the notebook.

In [ ]:
import asyncio
import os
from typing import Annotated
from dotenv import load_dotenv

# Microsoft Agent Framework core
from agent_framework import ChatAgent, ChatMessage, TextContent, Role

# Azure integrations
from agent_framework.azure import AzureOpenAIChatClient, AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential, AzureCliCredential

# Tool decorators
from agent_framework import ai_function


load_dotenv(override=True)
print("✓ Libraries imported successfully")

Configure environment variables. You can use either Azure OpenAI or Azure AI Foundry endpoints.

In [ ]:
# Check required environment variables
print("Checking environment variables...")

# For Azure OpenAI
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_version = os.getenv("AZURE_OPENAI_API_VERSION")

# For Azure AI Foundry
azure_project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
azure_ai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if azure_openai_endpoint and azure_openai_deployment:
    print("✓ Azure OpenAI configuration found")
    SERVICE_TYPE = "azure_openai"
else:
    print("❌ Missing required environment variables!")
    print("\nFor Azure OpenAI, set:")
    print("  - AZURE_OPENAI_ENDPOINT")
    print("  - AZURE_OPENAI_API_KEY")
    print("  - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    print("\nOR for Azure AI Foundry, set:")
    print("  - AZURE_AI_PROJECT_ENDPOINT")
    print("  - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    
print(f"\nUsing service: {SERVICE_TYPE}")
azure_openai_deployment

# Case 1: Basic Agent Creation and Execution

In Microsoft Agent Framework, creating an agent is straightforward. Unlike Semantic Kernel which requires a Kernel instance, Agent Framework agents are created directly from chat clients.

## 🧪 Case 1.1: Simple Agent with Azure OpenAI
---
Create a basic ChatAgent using AzureOpenAIChatClient. The agent handles thread management automatically.

In [ ]:
# Create a chat client and agent with Azure OpenAI
azure_openai_chat_client = AzureOpenAIChatClient(
    deployment_name=azure_openai_deployment,
    endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    api_version=azure_openai_version
)

# Create an agent - no Kernel needed!
agent = ChatAgent(
    chat_client=azure_openai_chat_client,
    instructions="You are a helpful assistant that provides concise, informative answers.",
    name="SimpleAssistant"
)

# Run the agent with a simple question
async def run_simple_agent():
    result = await agent.run("Tell me about Microsoft in 3 sentences.")
    print(f"Agent: {result.text}")

await run_simple_agent()

## 🧪 Case 1.2: Streaming Response
---
Agent Framework supports streaming responses for real-time output.

In [ ]:
async def run_streaming_agent():
    print("Agent: ", end="", flush=True)
    async for update in agent.run_stream("Tell me about Agentic AI in 3 sentences."):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming

await run_streaming_agent()

## 🧪 Case 1.3: Multi-turn Conversation with Thread
---
Agents automatically maintain conversation context using threads.

In [ ]:
async def multi_turn_conversation():
    
    # Create a new thread.
    thread = agent.get_new_thread()
    
    response1 = await agent.run("My name is Alex. I like programming in Python.", thread=thread)
    print(f"Agent: {response1.text}\n")
    
    # Get the thread from the result
    serialized = await thread.serialize()

    # Later, deserialize and continue conversation
    new_thread = await agent.deserialize_thread(serialized)
    
    # Second message - uses the same thread to maintain context
    response2 = await agent.run("What is my name?", thread=new_thread)
    print(f"Agent: {response2.text}\n")
    
    # Third message - agent remembers both previous exchanges
    response3 = await agent.run("What programming language did I say I like?", thread=new_thread)
    print(f"Agent: {response3.text}")

await multi_turn_conversation()

# Case 2: Agents with Custom Tools/Functions
---
Agent Framework makes it easy to extend agents with custom functions using the @ai_function decorator.
Define custom functions and register them with the agent. No plugin wrappers needed!

* The @ai_function decorator is used to define Custom Functions that agents can call as simple Python functions, attaching metadata (name, description, parameter/return schemas, etc.) to register them with the agent. This allows extending agent capabilities with just function definitions, without separate plugin wrappers or complex class inheritance.

### 🧪 Case 2.1: Azure Open AI Client with Custom Function Tools

In [ ]:
# Define custom functions using @ai_function decorator
@ai_function(description="Get the current weather for a location")
def get_weather(location: Annotated[str, "The city name"]) -> str:
    """Returns mock weather data for the given location."""
    # In a real scenario, this would call a weather API
    weather_data = {
        "Seoul": "Sunny, 22°C",
        "New York": "Cloudy, 18°C",
        "London": "Rainy, 15°C",
        "Tokyo": "Clear, 20°C",
        "Bangkok": "Hot, 30°C"
    }
    return weather_data.get(location, f"Weather data not available for {location}")

@ai_function(description="Get the current time in a timezone")
def get_time(timezone: Annotated[str, "Timezone (e.g., 'UTC', 'EST', 'KST', 'SGT')"]) -> str:
    """Returns mock time for the given timezone."""
    from datetime import datetime
    # Simplified for demo - in production use proper timezone handling
    return f"Current time in {timezone}: {datetime.now().strftime('%H:%M:%S')}"

# Create an agent with these tools
weather_agent = ChatAgent(
    chat_client=azure_openai_chat_client,
    instructions="You are a helpful assistant that can provide weather and time information.",
    name="WeatherAssistant",
    tools=[get_weather, get_time]
)

# Test the agent with tool calls
async def test_tools():
    result = await weather_agent.run("How is the weather in Bangkok?")
    print(f"Agent: {result.text}\n")
    
    result2 = await weather_agent.run("What time is it in SGT?")
    print(f"Agent: {result2.text}")

await test_tools()

# Case 4: Enterprise Tools with Azure AI Agent Client
---
Microsoft Agent Framework provides powerful enterprise-grade hosted tools that integrate seamlessly with Azure services. These tools enable agents to perform advanced operations like web search, document retrieval, and external integrations.

## Hosted Tools Comparison

| Tool | Purpose | Service | Key Features |
|------|---------|---------|--------------|
| **HostedFileSearchTool** | Document/Data Search | Azure AI Search | ✅ Index-based search<br>✅ Semantic search<br>✅ Configurable ranking |
| **HostedWebSearchTool** | Real-time Web Search | Bing Grounding | ✅ Current information<br>✅ Source citations<br>✅ Web grounding |
| **Local MCP** | Custom Integrations | Local Server | ✅ Full control<br>✅ Custom logic<br>✅ Local resources |
| **Hosted MCP** | External Services | Remote Server | ✅ Third-party APIs<br>✅ Scalable<br>✅ Managed infrastructure |

### Prerequisites for Practicing [Case 4]:

Log in to your Azure account. (Please perform Azure CLI login in the terminal first.)
- az login --use-device-code
- az account set --subscription "your-subscription-id"

### Check current login status and subscription
- az account show --output table

**For Azure AI Search (Case 4.1):**
- Azure AI Search service with indexed data
- Search connection configured in Azure AI Project
- Index name (e.g., "hotels-sample-index")

**For Bing Grounding (Case 4.2):**
- Bing Grounding connection in Azure AI Project
- `BING_CONNECTION_NAME` or `BING_CONNECTION_ID` environment variable

**For MCP Tools (Case 4.3 & 4.4):**
- Understanding of Model Context Protocol (MCP)
- MCP server configuration (local or hosted)

**In this training, we will proceed with Case 4.2 Bing Grounding. For the source code of other cases, please refer to the [original GitHub repository's Python code](https://github.com/Azure/agent-innovator-lab/blob/main/0_basic-agent/AgentFramework/1_basic-concept-with-msaf.ipynb).**

* Note: For those who can not create Bing Grounding resource in Azure, please skip Case 4.1 and proceed to Case 4.3.

## 🧪 Case 4.2: Bing Grounding with HostedWebSearchTool
---
Demonstrates using Bing Search API for web grounding to answer questions with real-time web data.

**Setup:**
- Azure AI Project with Bing Search connection
- Environment variables: `AZURE_AI_PROJECT_ENDPOINT`, `BING_CONNECTION_NAME`

**Key Modules:**
- `HostedWebSearchTool` from `agent_framework.azure_ai.tools`
- Provides grounded responses with web search results and citations

In [ ]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIAgentClient, AzureOpenAIChatClient
from agent_framework import HostedWebSearchTool
from agent_framework import ChatAgent

azure_project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
azure_ai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

async def bing_grounding_demo():
    """
    Demonstrates using Bing Search API for web grounding to answer questions with real-time web data.
    """
    
    try:
        # Get Azure AI Project endpoint
        bing_connection_id = os.environ.get("BING_GROUNDING_CONNECTION_ID")
        
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        # Create web search tool with Bing connection
        web_search_tool = HostedWebSearchTool(
            
            additional_properties={
                "connection_id": bing_connection_id,
                "top_k": 3,  # Get top 3 search results
            }
        )
        
        azure_ai_agent_client = AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        )
        
        # Create agent with web search capability
        agent = ChatAgent(
            #chat_client=azure_openai_chat_client,
            chat_client=azure_ai_agent_client,
            name="Web Research Assistant",
            instructions="You are a helpful research assistant. Use web search to find current information and provide grounded answers with citations.",
            tools=[web_search_tool]
        )
        
        # Test questions that require web search
        USER_INPUTS = [
            "Tell me what day it is today and What is the latest exciting news in Bangkok?"
        ]
        
        print("=== Azure AI Agent with Bing Grounding Search ===")
        print("This agent can search the web for current information and provide grounded answers.\n")
        
        # Simulate conversation with the agent
        for user_input in USER_INPUTS:
            print(f"User: {user_input}")
            print("Agent: ", end="", flush=True)
            
            # Stream the response for better user experience
            async for chunk in agent.run_stream(user_input):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            print("\n" + "=" * 50 + "\n")
        
        print("Web search conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        # Cleanup: Close Azure AI Agent client if it was created
        try:
            await azure_ai_agent_client.close()    
        except Exception as cleanup_error:
            print(f"Cleanup warning: {cleanup_error}")

await bing_grounding_demo()

## 🧪 Case 4.3: Local MCP Server Integration
---
Demonstrates connecting to a local Model Context Protocol (MCP) server to extend agent capabilities with custom tools.

### Setup:

- Local MCP server running (e.g., file system tools, database connectors)
- MCP server URL or configuration
- Environment variable: AZURE_OPENAI_ENDPOINT

### Key Modules:

- Discovers and registers tools from MCP server dynamically
- Works with any MCP-compliant server implementation

In [ ]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureOpenAIChatClient, AzureAIAgentClient
from agent_framework import ChatAgent, MCPStreamableHTTPTool


async def local_mcp_demo():
    """
    Demonstrates connecting to a local Model Context Protocol (MCP) server 
    to extend agent capabilities with custom tools.
    
    This example shows how to use Microsoft Learn MCP server with Azure AI Agent.
    """
    
    try:
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        print("=== Azure AI Agent with Local MCP Server ===")
        print("This agent can use MCP tools to access Microsoft documentation.\n")
        
        # Connect to local MCP server and create agent with MCP tools
        async with (
            AzureAIAgentClient(
                async_credential=credential,
                project_endpoint=azure_project_endpoint,
                model_deployment_name=azure_ai_deployment
            ) as azure_ai_agent_client,
            MCPStreamableHTTPTool(
                name="Local Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
            ) as mcp_server
        ):
            # Create agent with MCP tools
            agent = ChatAgent(
                chat_client=azure_ai_agent_client,
                name="Local MCP Assistant(AzureAIAgentClient)",
                instructions="You are a helpful assistant that can help with Microsoft documentation questions using MCP tools.",
                tools=mcp_server,
                store=True
            )
            
            # Test questions that use MCP tools
            USER_INPUTS = [
                "How to create an Azure storage account using az cli? response with a simple answer and an example.",
                "What is Microsoft Agent Framework? response with key concepts and a main python code example."
            ]
            
            # Simulate conversation with the agent
            for user_input in USER_INPUTS:
                print(f"User: {user_input}")
                print("Agent: ", end="", flush=True)
                
                # Stream the response for better user experience
                async for chunk in agent.run_stream(user_input):
                    if chunk.text:
                        print(chunk.text, end="", flush=True)
                print("\n" + "=" * 50 + "\n")
            
            print("MCP tool conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback
        traceback.print_exc()

await local_mcp_demo()

## 📊 Case 4 Comparison: Enterprise Tools

| Feature | AI Search (4.1) | Bing Grounding (4.2) | Local MCP (4.3) | Hosted MCP (4.4) |
|---------|----------------|---------------------|----------------|------------------|
| **Tool Type** | `HostedFileSearchTool` | `HostedWebSearchTool` | `MCPClient` | `HostedMCPTool` |
| **Client Type** | `AzureAIAgentClient` | `AzureAIAgentClient` | `AzureOpenAIChatClient` | `AzureAIAgentClient` |
| **Data Source** | Azure AI Search vector stores | Bing Search API | Local MCP server process | Azure-hosted MCP server |
| **Setup Complexity** | Medium (requires vector store) | Low (requires Bing connection) | Medium (requires local server) | Low (requires Azure connection) |
| **Use Cases** | Internal document search, knowledge base | Real-time web information | Custom local tools, file system | Enterprise-grade custom tools |
| **Authentication** | `DefaultAzureCredential` | `DefaultAzureCredential` | `AzureCliCredential` | `DefaultAzureCredential` |
| **Grounding** | Citations from indexed docs | Web citations and sources | N/A (custom tool output) | Depends on MCP server |
| **Scalability** | High (Azure managed) | High (Azure managed) | Low (local process) | High (Azure managed) |
| **Customization** | Limited to indexed data | Limited to web search | Full (custom MCP tools) | Full (custom MCP tools) |
| **Cost** | AI Search + AI Project | Bing API + AI Project | Azure OpenAI only | MCP hosting + AI Project |

**Key Takeaways:**
- AI Search (4.1): Best for searching through your own documents and knowledge bases with semantic search
- Bing Grounding (4.2): Best for real-time web information with source citations
- Local MCP (4.3): Best for development and testing with custom tools on local machine
- Hosted MCP (4.4): Best for production deployment of custom tools with enterprise security and scalability